In [1]:
# Import Packages
from pathlib import Path
from os import access
import geopandas as gpd
from pandas.core.frame import DataFrame
import rioxarray as rio
import xarray as xr
import rasterio
from shapely.geometry import mapping, Polygon
import matplotlib.pyplot as plt
import pandas as pd
from xarray.core.computation import join_dict_keys
from xarray.core.parallel import dataset_to_dataarray
from xarray.core.utils import V
import xesmf as xe
import cartopy.crs as ccrs
import numpy as np
# import pymannkendall as mk
# import xclim
# from xclim.indicators import atmos
# from xclim import indices

# to check for autocorrelation
from statsmodels.graphics.tsaplots import plot_acf , plot_pacf
from pandas.plotting import autocorrelation_plot

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import QuantileTransformer

# Deep Learning modules
import tensorflow as tf 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Reshape, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

2022-11-19 13:34:30.956422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# keep attributes of all datasets

xr.set_options(keep_attrs=True)

In [3]:
from dask.distributed import Client, LocalCluster

In [4]:
# file directory 
inDir = "/datadrive/project/climate/dl_downscaling/data_raw/gpm_imerge_raw/"
inDir2 = "/datadrive/project/climate/dl_downscaling/data_raw/era5_downscale_dataset/"
outDir = "/datadrive/project/climate/dl_downscaling/output/"

In [5]:
# Path to Geometry files

PATH_TO_SHAPEFILE = '/datadrive/project/climate/dl_downscaling/data_raw/nigerbasin_merged_polygon/hybas_nigerbasin_lev3.shp'

In [6]:
# open the shapefiles with geopandas
basin_polygon = gpd.read_file(PATH_TO_SHAPEFILE)
basin_polygon

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,SIDE,LAKE,ENDO,COAST,ORDER_,SORT,geometry
0,1030022420,0,1030022420,1030022420,0.0,0.0,2122995.6,2122995.6,142,M,0,0,0,1,17,"POLYGON ((-7.20833 9.71250, -7.20869 9.70591, ..."


In [7]:
# climatology filenames
pr = "gpm_africa_daily_transposed_20000930_20210930.nc"

# windspeed = 'windspeed_10m_day_1979_2020.nc'
tas = 'era5_tas_day1979_2021_transposed.nc'
tasmax = 'era5_tmax_day1979_2021.nc'
tasmin = 'era5_tmin_day1979_2021.nc'
huss = 'era5_specific_humidity_1000hpa_resample_2000_2020_transposed.nc'
zg500 = 'era5_geopotential_500hpa_resample_2000_2020_transposed.nc'
zg1000 = 'geopotential_1000hpa_daily_2000_2020.nc'
era5_pr = 'era5_pr_daily_transposed_dim_1979_2021.nc'
psl = 'mean_sea_level_pressure_daily_2000_2020.nc'
uas = 'uwind_10m_day_1979_2020.nc'
vas = 'vwind_10m_day_1979_2020.nc'

In [8]:
ds = xr.open_dataset(inDir + pr, chunks={'lat': 25, 'lon': 25, 'time': -1})
ds = ds.transpose("time", "lat", "lon")
ds = ds.sel(time=slice('2001-01-01', '2020-12-31'))
ds 

<xarray.Dataset>
Dimensions:  (lat: 281, lon: 361, time: 7305)
Coordinates:
  * lat      (lat) float32 1.95 2.05 2.15 2.25 2.35 ... 29.65 29.75 29.85 29.95
  * lon      (lon) float32 -18.05 -17.95 -17.85 -17.75 ... 17.75 17.85 17.95
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
Data variables:
    pr       (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [9]:
ds_tas = xr.open_dataset(inDir2 + tas, chunks={'lat': 25, 'lon': 25, 'time': -1})
ds_tas = ds_tas.transpose("time", "lat", "lon")
ds_tas = ds_tas.sel(time=slice('2001-01-01', '2020-12-31'))
ds_tas = ds_tas.rename({'tas_day':'tas'})
ds_tas

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    tas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [10]:
ds_tasmax = xr.open_dataset(inDir2 + tasmax, chunks={'latitude': 25, 'longitude': 25, 'time': -1})
ds_tasmax = ds_tasmax.sel(time=slice('2001-01-01', '2020-12-31'))
ds_tasmax = ds_tasmax.rename({'tmax_day':'tasmax'})
ds_tasmax = ds_tasmax.rename({'longitude':'lon'})
ds_tasmax = ds_tasmax.rename({'latitude':'lat'})
ds_tasmax

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [11]:
ds_tasmin = xr.open_dataset(inDir2 + tasmin, chunks={'latitude': 25, 'longitude': 25, 'time': -1})
ds_tasmin = ds_tasmin.sel(time=slice('2001-01-01', '2020-12-31'))
ds_tasmin = ds_tasmin.rename({'tmin_day':'tasmin'})
ds_tasmin = ds_tasmin.rename({'longitude':'lon'})
ds_tasmin = ds_tasmin.rename({'latitude':'lat'})
ds_tasmin 

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    tasmin   (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [12]:
ds_huss = xr.open_dataset(inDir2 + huss, chunks={'lat': 25, 'lon': 25, 'time': -1})
ds_huss = ds_huss.transpose("time", "lat", "lon")
ds_huss = ds_huss.sel(time=slice('2001-01-01', '2020-12-31'))
# ds_huss = ds_huss.rename({'longitude':'lon'})
# ds_huss = ds_huss.rename({'latitude':'lat'})
ds_huss 

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    q        (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [13]:
ds_zg500 = xr.open_dataset(inDir2 + zg500, chunks={'lat': 25, 'lon': 25, 'time': -1})
ds_zg500 = ds_zg500.transpose("time", "lat", "lon")
ds_zg500 = ds_zg500.sel(time=slice('2001-01-01', '2020-12-31'))
ds_zg500 = ds_zg500.rename({'z':'z500'})
# ds_zg500 = ds_zg500.rename({'longitude':'lon'})
# ds_zg500 = ds_zg500.rename({'latitude':'lat'})
ds_zg500

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    z500     (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [14]:
ds_zg1000 = xr.open_dataset(inDir2 + zg1000, chunks={'latitude': 25, 'longitude': 25, 'time': -1})
ds_zg1000 = ds_zg1000.sel(time=slice('2001-01-01', '2020-12-31'))
ds_zg1000 = ds_zg1000.rename({'z':'z1000'})
ds_zg1000 = ds_zg1000.rename({'longitude':'lon'})
ds_zg1000 = ds_zg1000.rename({'latitude':'lat'})
ds_zg1000

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    z1000    (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [15]:
ds_psl = xr.open_dataset(inDir2 + psl, chunks={'latitude': 25, 'longitude': 25, 'time': -1})
ds_psl = ds_psl.sel(time=slice('2001-01-01', '2020-12-31'))
ds_psl = ds_psl.rename({'msl':'psl'})
ds_psl = ds_psl.rename({'longitude':'lon'})
ds_psl = ds_psl.rename({'latitude':'lat'})
ds_psl

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    psl      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [16]:
ds_uas = xr.open_dataset(inDir2 + uas, chunks={'latitude': 25, 'longitude': 25, 'time': -1})
ds_uas = ds_uas.sel(time=slice('2001-01-01', '2020-12-31'))
ds_uas = ds_uas.rename({'u10':'uas'})
ds_uas = ds_uas.rename({'longitude':'lon'})
ds_uas = ds_uas.rename({'latitude':'lat'})
ds_uas

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    uas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-09-16 08:43:49 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [17]:
ds_vas = xr.open_dataset(inDir2 + vas, chunks={'latitude': 25, 'longitude': 25, 'time': -1})
ds_vas = ds_vas.sel(time=slice('2001-01-01', '2020-12-31'))
ds_vas = ds_vas.rename({'v10':'vas'})
ds_vas = ds_vas.rename({'longitude':'lon'})
ds_vas = ds_vas.rename({'latitude':'lat'})
ds_vas

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    vas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-09-17 14:36:07 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [18]:
ds_era5_pr = xr.open_dataset(inDir2 + era5_pr, chunks={'lat': 25, 'lon': 25, 'time': -1})
ds_era5_pr = ds_era5_pr.transpose("time", "lat", "lon")
ds_era5_pr = ds_era5_pr.sel(time=slice('2001-01-01', '2020-12-31'))
ds_era5_pr = ds_era5_pr.rename({'pr':'era5_pr'})
ds_era5_pr

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    era5_pr  (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [19]:
# Standardized GPM IMERGE Precipitation by month

In [20]:
# Standardized GPM IMERGE Precipitation by month

In [21]:
ds_gb_mnt = ds.groupby('time.month')
ds_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [22]:
# get the groupby month mean of years 2001 to 2020
ds_gbmonth_mean = ds.groupby('time.month').mean('time')
ds_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 281, lon: 361)
Coordinates:
  * lat      (lat) float32 1.95 2.05 2.15 2.25 2.35 ... 29.65 29.75 29.85 29.95
  * lon      (lon) float32 -18.05 -17.95 -17.85 -17.75 ... 17.75 17.85 17.95
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    pr       (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [23]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_gbmonth_std = ds.groupby('time.month').std('time')
ds_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 281, lon: 361)
Coordinates:
  * lat      (lat) float32 1.95 2.05 2.15 2.25 2.35 ... 29.65 29.75 29.85 29.95
  * lon      (lon) float32 -18.05 -17.95 -17.85 -17.75 ... 17.75 17.85 17.95
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    pr       (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [24]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_month_anom = ds_gb_mnt - ds_gbmonth_mean
ds_month_anom

<xarray.Dataset>
Dimensions:  (lat: 281, lon: 361, time: 7305)
Coordinates:
  * lat      (lat) float32 1.95 2.05 2.15 2.25 2.35 ... 29.65 29.75 29.85 29.95
  * lon      (lon) float32 -18.05 -17.95 -17.85 -17.75 ... 17.75 17.85 17.95
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    pr       (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [25]:
ds_anom_gb_mnt = ds_month_anom.groupby('time.month')
ds_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [26]:
# standardize monthly anomalous dataarray
ds_standardized = ds_anom_gb_mnt / ds_gbmonth_std
ds_standardized = ds_standardized.drop('month', dim=None)
ds_standardized

<xarray.Dataset>
Dimensions:  (lat: 281, lon: 361, time: 7305)
Coordinates:
  * lat      (lat) float32 1.95 2.05 2.15 2.25 2.35 ... 29.65 29.75 29.85 29.95
  * lon      (lon) float32 -18.05 -17.95 -17.85 -17.75 ... 17.75 17.85 17.95
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
Data variables:
    pr       (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [27]:
# Standardized Reanalysis variables by month

# Standardize Mean Temperature

In [28]:
ds_tas_gb_mnt = ds_tas.groupby('time.month')
ds_tas_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [29]:
# get the groupby month mean of years 2001 to 2020
ds_tas_gbmonth_mean = ds_tas.groupby('time.month').mean('time')
ds_tas_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tas      (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [30]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_tas_gbmonth_std = ds_tas.groupby('time.month').std('time')
ds_tas_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tas      (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [31]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_tas_month_anom = ds_tas_gb_mnt - ds_tas_gbmonth_mean
ds_tas_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    tas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [32]:
ds_tas_anom_gb_mnt = ds_tas_month_anom.groupby('time.month')
ds_tas_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [33]:
# standardize monthly anomalous dataarray
ds_tas_standardized = ds_tas_anom_gb_mnt / ds_tas_gbmonth_std
ds_tas_standardized = ds_tas_standardized.drop('month', dim=None)
ds_tas_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    tas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [34]:
# Standadized Maximum Temperature

In [35]:
ds_tasmax_gb_mnt = ds_tasmax.groupby('time.month')
ds_tasmax_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [36]:
# get the groupby month mean of years 2001 to 2020
ds_tasmax_gbmonth_mean = ds_tasmax.groupby('time.month').mean('time')
ds_tasmax_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tasmax   (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [37]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_tasmax_gbmonth_std = ds_tasmax.groupby('time.month').std('time')
ds_tasmax_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tasmax   (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [38]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_tasmax_month_anom = ds_tasmax_gb_mnt - ds_tasmax_gbmonth_mean
ds_tasmax_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [39]:
ds_tasmax_anom_gb_mnt = ds_tasmax_month_anom.groupby('time.month')
ds_tasmax_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [40]:
# standardize monthly anomalous dataarray
ds_tasmax_standardized = ds_tasmax_anom_gb_mnt / ds_tasmax_gbmonth_std
ds_tasmax_standardized = ds_tasmax_standardized.drop('month', dim=None)
ds_tasmax_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    tasmax   (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [41]:
# Standardize Minimum Temperature

In [42]:
ds_tasmin_gb_mnt = ds_tasmin.groupby('time.month')
ds_tasmin_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [43]:
# get the groupby month mean of years 2001 to 2020
ds_tasmin_gbmonth_mean = ds_tasmin.groupby('time.month').mean('time')
ds_tasmin_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tasmin   (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [44]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_tasmin_gbmonth_std = ds_tasmin.groupby('time.month').std('time')
ds_tasmin_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    tasmin   (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [45]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_tasmin_month_anom = ds_tasmin_gb_mnt - ds_tasmin_gbmonth_mean
ds_tasmin_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    tasmin   (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [46]:
ds_tasmin_anom_gb_mnt = ds_tasmin_month_anom.groupby('time.month')
ds_tasmin_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [47]:
# standardize monthly anomalous dataarray
ds_tasmin_standardized = ds_tasmin_anom_gb_mnt / ds_tasmin_gbmonth_std
ds_tasmin_standardized = ds_tasmin_standardized.drop('month', dim=None)
ds_tasmin_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    tasmin   (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [48]:
# Standardized Specific Humidity

In [49]:
ds_huss_gb_mnt = ds_huss.groupby('time.month')
ds_huss_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [50]:
# get the groupby month mean of years 2001 to 2020
ds_huss_gbmonth_mean = ds_huss.groupby('time.month').mean('time')
ds_huss_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    q        (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [51]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_huss_gbmonth_std = ds_huss.groupby('time.month').std('time')
ds_huss_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    q        (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [52]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_huss_month_anom = ds_huss_gb_mnt - ds_huss_gbmonth_mean
ds_huss_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    q        (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [53]:
ds_huss_anom_gb_mnt = ds_huss_month_anom.groupby('time.month')
ds_huss_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [54]:
# standardize monthly anomalous dataarray
ds_huss_standardized = ds_huss_anom_gb_mnt / ds_huss_gbmonth_std
ds_huss_standardized = ds_huss_standardized.drop('month', dim=None)
ds_huss_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    q        (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [55]:
# Standardized Geopotential at 500hpa

In [56]:
ds_zg500_gb_mnt = ds_zg500.groupby('time.month')
ds_zg500_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [57]:
# get the groupby month mean of years 2001 to 2020
ds_zg500_gbmonth_mean = ds_zg500.groupby('time.month').mean('time')
ds_zg500_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    z500     (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [58]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_zg500_gbmonth_std = ds_zg500.groupby('time.month').std('time')
ds_zg500_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    z500     (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [59]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_zg500_month_anom = ds_zg500_gb_mnt - ds_zg500_gbmonth_mean
ds_zg500_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    z500     (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [60]:
ds_zg500_anom_gb_mnt = ds_zg500_month_anom.groupby('time.month')
ds_zg500_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [61]:
# standardize monthly anomalous dataarray
ds_zg500_standardized = ds_zg500_anom_gb_mnt / ds_zg500_gbmonth_std
ds_zg500_standardized = ds_zg500_standardized.drop('month', dim=None)
ds_zg500_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    z500     (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [62]:
# Standardized Geopotential at 1000hpa

In [63]:
ds_zg1000_gb_mnt = ds_zg1000.groupby('time.month')
ds_zg1000_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [64]:
# get the groupby month mean of years 2001 to 2020
ds_zg1000_gbmonth_mean = ds_zg1000.groupby('time.month').mean('time')
ds_zg1000_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    z1000    (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [65]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_zg1000_gbmonth_std = ds_zg1000.groupby('time.month').std('time')
ds_zg1000_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    z1000    (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [66]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_zg1000_month_anom = ds_zg1000_gb_mnt - ds_zg1000_gbmonth_mean
ds_zg1000_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    z1000    (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [67]:
ds_zg1000_anom_gb_mnt = ds_zg1000_month_anom.groupby('time.month')
ds_zg1000_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [68]:
# standardize monthly anomalous dataarray
ds_zg1000_standardized = ds_zg1000_anom_gb_mnt / ds_zg1000_gbmonth_std
ds_zg1000_standardized = ds_zg1000_standardized.drop('month', dim=None)
ds_zg1000_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    z1000    (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [69]:
# Standardized Mean Sea Level Pressure

In [70]:
ds_psl_gb_mnt = ds_psl.groupby('time.month')
ds_psl_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [71]:
# get the groupby month mean of years 2001 to 2020
ds_psl_gbmonth_mean = ds_psl.groupby('time.month').mean('time')
ds_psl_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    psl      (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [72]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_psl_gbmonth_std = ds_psl.groupby('time.month').std('time')
ds_psl_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    psl      (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [73]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_psl_month_anom = ds_psl_gb_mnt - ds_psl_gbmonth_mean
ds_psl_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    psl      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [74]:
ds_psl_anom_gb_mnt = ds_psl_month_anom.groupby('time.month')
ds_psl_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [75]:
# standardize monthly anomalous dataarray
ds_psl_standardized = ds_psl_anom_gb_mnt / ds_psl_gbmonth_std
ds_psl_standardized = ds_psl_standardized.drop('month', dim=None)
ds_psl_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    psl      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [76]:
# Standardized Eastward Wind

In [77]:
ds_uas_gb_mnt = ds_uas.groupby('time.month')
ds_uas_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [78]:
# get the groupby month mean of years 2001 to 2020
ds_uas_gbmonth_mean = ds_uas.groupby('time.month').mean('time')
ds_uas_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    uas      (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-09-16 08:43:49 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [79]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_uas_gbmonth_std = ds_uas.groupby('time.month').std('time')
ds_uas_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    uas      (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-09-16 08:43:49 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [80]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_uas_month_anom = ds_uas_gb_mnt - ds_uas_gbmonth_mean
ds_uas_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    uas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [81]:
ds_uas_anom_gb_mnt = ds_uas_month_anom.groupby('time.month')
ds_uas_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [82]:
# standardize monthly anomalous dataarray
ds_uas_standardized = ds_uas_anom_gb_mnt / ds_uas_gbmonth_std
ds_uas_standardized = ds_uas_standardized.drop('month', dim=None)
ds_uas_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    uas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [83]:
# Standardized Northward Wind

In [84]:
ds_vas_gb_mnt = ds_vas.groupby('time.month')
ds_vas_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [85]:
# get the groupby month mean of years 2001 to 2020
ds_vas_gbmonth_mean = ds_vas.groupby('time.month').mean('time')
ds_vas_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    vas      (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-09-17 14:36:07 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [86]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_vas_gbmonth_std = ds_vas.groupby('time.month').std('time')
ds_vas_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    vas      (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2021-09-17 14:36:07 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [87]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_vas_month_anom = ds_vas_gb_mnt - ds_vas_gbmonth_mean
ds_vas_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    vas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [88]:
ds_vas_anom_gb_mnt = ds_vas_month_anom.groupby('time.month')
ds_vas_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [89]:
# standardize monthly anomalous dataarray
ds_vas_standardized = ds_vas_anom_gb_mnt / ds_vas_gbmonth_std
ds_vas_standardized = ds_vas_standardized.drop('month', dim=None)
ds_vas_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    vas      (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [90]:
# Standardized Era5 Precipitation

In [91]:
ds_era5_pr_gb_mnt = ds_era5_pr.groupby('time.month')
ds_era5_pr_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [92]:
# get the groupby month mean of years 2001 to 2020
ds_era5_pr_gbmonth_mean = ds_era5_pr.groupby('time.month').mean('time')
ds_era5_pr_gbmonth_mean

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    era5_pr  (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [93]:
# get the groupby monthly standard deviation of years 2010 to 2014
ds_era5_pr_gbmonth_std = ds_era5_pr.groupby('time.month').std('time')
ds_era5_pr_gbmonth_std

<xarray.Dataset>
Dimensions:  (month: 12, lat: 113, lon: 145)
Coordinates:
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    era5_pr  (month, lat, lon) float32 dask.array<chunksize=(12, 25, 25), meta=np.ndarray>

In [94]:
# transform data to grouped monthly anomalous with calibration of 2001 to 2014
ds_era5_pr_month_anom = ds_era5_pr_gb_mnt - ds_era5_pr_gbmonth_mean
ds_era5_pr_month_anom

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
    month    (time) int64 1 1 1 1 1 1 1 1 1 1 ... 12 12 12 12 12 12 12 12 12 12
Data variables:
    era5_pr  (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [95]:
ds_era5_pr_anom_gb_mnt = ds_era5_pr_month_anom.groupby('time.month')
ds_era5_pr_anom_gb_mnt

DatasetGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [96]:
# standardize monthly anomalous dataarray
ds_era5_pr_standardized = ds_era5_pr_anom_gb_mnt / ds_era5_pr_gbmonth_std
ds_era5_pr_standardized = ds_era5_pr_standardized.drop('month', dim=None)
ds_era5_pr_standardized

<xarray.Dataset>
Dimensions:  (time: 7305, lon: 145, lat: 113)
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2020-12-31
  * lon      (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.25 17.5 17.75 18.0
  * lat      (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 3.0 2.75 2.5 2.25 2.0
Data variables:
    era5_pr  (time, lat, lon) float32 dask.array<chunksize=(7305, 25, 25), meta=np.ndarray>

In [97]:
from pickle import dump

# create directory for dataset transformers
DATA_PATH = Path("/datadrive/project/")


scaler_path = DATA_PATH / "scaler_dir"
scaler_path.mkdir(exist_ok=True)

In [98]:
# Save ERA5 xarray scaler dataset (2001 to 2020)

ds_tas_gbmonth_mean = ds_tas_gbmonth_mean.rename({'tas':'tas_mean'})
ds_tas_gbmonth_std = ds_tas_gbmonth_std.rename({'tas':'tas_std'})

ds_tasmax_gbmonth_mean = ds_tasmax_gbmonth_mean.rename({'tasmax':'tasmax_mean'})
ds_tasmax_gbmonth_std = ds_tasmax_gbmonth_std.rename({'tasmax':'tasmax_std'})

ds_tasmin_gbmonth_mean = ds_tasmin_gbmonth_mean.rename({'tasmin':'tasmin_mean'})
ds_tasmin_gbmonth_std = ds_tasmin_gbmonth_std.rename({'tasmin':'tasmin_std'})

ds_huss_gbmonth_mean = ds_huss_gbmonth_mean.rename({'q':'q_mean'})
ds_huss_gbmonth_std = ds_huss_gbmonth_std.rename({'q':'q_std'})

ds_zg500_gbmonth_mean = ds_zg500_gbmonth_mean.rename({'z500':'z500_mean'})
ds_zg500_gbmonth_std = ds_zg500_gbmonth_std.rename({'z500':'z500_std'})

ds_zg1000_gbmonth_mean = ds_zg1000_gbmonth_mean.rename({'z1000':'z1000_mean'})
ds_zg1000_gbmonth_std = ds_zg1000_gbmonth_std.rename({'z1000':'z1000_std'})

ds_psl_gbmonth_mean = ds_psl_gbmonth_mean.rename({'psl':'psl_mean'})
ds_psl_gbmonth_std = ds_psl_gbmonth_std.rename({'psl':'psl_std'})

ds_uas_gbmonth_mean = ds_uas_gbmonth_mean.rename({'uas':'uas_mean'})
ds_uas_gbmonth_std = ds_uas_gbmonth_std.rename({'uas':'uas_std'})

ds_vas_gbmonth_mean = ds_vas_gbmonth_mean.rename({'vas':'vas_mean'})
ds_vas_gbmonth_std = ds_vas_gbmonth_std.rename({'vas':'vas_std'})

ds_era5_pr_gbmonth_mean = ds_era5_pr_gbmonth_mean.rename({'era5_pr':'era5_pr_mean'})
ds_era5_pr_gbmonth_std = ds_era5_pr_gbmonth_std.rename({'era5_pr':'era5_pr_std'})

In [99]:
da_tas_gbmonth_mean = ds_tas_gbmonth_mean.tas_mean.load()
da_tas_gbmonth_std = ds_tas_gbmonth_std.tas_std.load()

da_tasmax_gbmonth_mean = ds_tasmax_gbmonth_mean.tasmax_mean.load()
da_tasmax_gbmonth_std = ds_tasmax_gbmonth_std.tasmax_std.load()

da_tasmin_gbmonth_mean = ds_tasmin_gbmonth_mean.tasmin_mean.load()
da_tasmin_gbmonth_std = ds_tasmin_gbmonth_std.tasmin_std.load()

da_huss_gbmonth_mean = ds_huss_gbmonth_mean.q_mean.load()
da_huss_gbmonth_std = ds_huss_gbmonth_std.q_std.load()

da_zg500_gbmonth_mean = ds_zg500_gbmonth_mean.z500_mean.load()
da_zg500_gbmonth_std = ds_zg500_gbmonth_std.z500_std.load()

da_zg1000_gbmonth_mean = ds_zg1000_gbmonth_mean.z1000_mean.load()
da_zg1000_gbmonth_std = ds_zg1000_gbmonth_std.z1000_std.load()

da_psl_gbmonth_mean = ds_psl_gbmonth_mean.psl_mean.load()
da_psl_gbmonth_std = ds_psl_gbmonth_std.psl_std.load()

da_uas_gbmonth_mean = ds_uas_gbmonth_mean.uas_mean.load()
da_uas_gbmonth_std = ds_uas_gbmonth_std.uas_std.load()

da_vas_gbmonth_mean = ds_vas_gbmonth_mean.vas_mean.load()
da_vas_gbmonth_std = ds_vas_gbmonth_std.vas_std.load()

da_era5_pr_gbmonth_mean = ds_era5_pr_gbmonth_mean.era5_pr_mean.load()
da_era5_pr_gbmonth_std = ds_era5_pr_gbmonth_std.era5_pr_std.load()

In [100]:
ds_era5_gbmonth_mean_std = da_tas_gbmonth_mean.to_dataset()
ds_era5_gbmonth_mean_std['tas_std'] = da_tas_gbmonth_std

ds_era5_gbmonth_mean_std['tasmax_mean'] = da_tasmax_gbmonth_mean
ds_era5_gbmonth_mean_std['tasmax_std'] = da_tasmax_gbmonth_std

ds_era5_gbmonth_mean_std['tasmin_mean'] = da_tasmin_gbmonth_mean
ds_era5_gbmonth_mean_std['tasmin_std'] = da_tasmin_gbmonth_std

ds_era5_gbmonth_mean_std['q_mean'] = da_huss_gbmonth_mean
ds_era5_gbmonth_mean_std['q_std'] = da_huss_gbmonth_std

ds_era5_gbmonth_mean_std['z500_mean'] = da_zg500_gbmonth_mean
ds_era5_gbmonth_mean_std['z500_std'] = da_zg500_gbmonth_std

ds_era5_gbmonth_mean_std['z1000_mean'] = da_zg1000_gbmonth_mean
ds_era5_gbmonth_mean_std['z1000_std'] = da_zg1000_gbmonth_std

ds_era5_gbmonth_mean_std['psl_mean'] = da_psl_gbmonth_mean
ds_era5_gbmonth_mean_std['psl_std'] = da_psl_gbmonth_std

ds_era5_gbmonth_mean_std['uas_mean'] = da_uas_gbmonth_mean
ds_era5_gbmonth_mean_std['uas_std'] = da_uas_gbmonth_std

ds_era5_gbmonth_mean_std['vas_mean'] = da_vas_gbmonth_mean
ds_era5_gbmonth_mean_std['vas_std'] = da_vas_gbmonth_std

ds_era5_gbmonth_mean_std['era5_pr_mean'] = da_era5_pr_gbmonth_mean
ds_era5_gbmonth_mean_std['era5_pr_std'] = da_era5_pr_gbmonth_std

ds_era5_gbmonth_mean_std

<xarray.Dataset>
Dimensions:       (lon: 145, lat: 113, month: 12)
Coordinates:
  * lon           (lon) float32 -18.0 -17.75 -17.5 -17.25 ... 17.5 17.75 18.0
  * lat           (lat) float32 30.0 29.75 29.5 29.25 29.0 ... 2.75 2.5 2.25 2.0
  * month         (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables: (12/20)
    tas_mean      (month, lat, lon) float32 18.2 18.17 18.13 ... 25.56 25.75
    tas_std       (month, lat, lon) float32 0.8866 0.8866 ... 0.8946 0.8398
    tasmax_mean   (month, lat, lon) float32 18.71 18.68 18.65 ... 31.02 30.9
    tasmax_std    (month, lat, lon) float32 0.8542 0.8566 0.8586 ... 1.526 1.459
    tasmin_mean   (month, lat, lon) float32 17.71 17.67 17.62 ... 21.82 22.16
    tasmin_std    (month, lat, lon) float32 0.9774 0.977 0.9732 ... 1.134 0.9935
    ...            ...
    uas_mean      (month, lat, lon) float32 -3.557 -3.536 ... 0.06356 0.04553
    uas_std       (month, lat, lon) float32 4.138 4.132 4.119 ... 0.6236 0.6098
    vas_mean      (month, lat, lon) float32 -2.27 -2.274 ... 0.6893 0.6938
    vas_std       (month, lat, lon) float32 3.475 3.452 3.447 ... 0.5975 0.6126
    era5_pr_mean  (month, lat, lon) float32 0.6298 0.5888 0.5558 ... 1.674 1.968
    era5_pr_std   (month, lat, lon) float32 1.859 1.588 1.568 ... 3.016 3.212

In [101]:
# Save GPM IMERGE xarray scaler dataset (2001 to 2020)
ds_gbmonth_mean = ds_gbmonth_mean.rename({'pr':'pr_mean'})
ds_gbmonth_std = ds_gbmonth_std.rename({'pr':'pr_std'})

da_gbmonth_mean = ds_gbmonth_mean.pr_mean.load()
da_gbmonth_std = ds_gbmonth_std.pr_std.load()

ds_gpm_gbmonth_mean_std = da_gbmonth_mean.to_dataset()
ds_gpm_gbmonth_mean_std['pr_std'] = da_gbmonth_std

ds_gpm_gbmonth_mean_std

<xarray.Dataset>
Dimensions:  (lat: 281, lon: 361, month: 12)
Coordinates:
  * lat      (lat) float32 1.95 2.05 2.15 2.25 2.35 ... 29.65 29.75 29.85 29.95
  * lon      (lon) float32 -18.05 -17.95 -17.85 -17.75 ... 17.75 17.85 17.95
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    pr_mean  (month, lat, lon) float32 17.21 16.85 16.36 ... 0.972 1.017 1.018
    pr_std   (month, lat, lon) float32 37.46 35.72 35.43 ... 3.984 4.054 4.019

In [102]:
ds_era5_gbmonth_mean_std.to_netcdf(scaler_path / 'era5_dl_downscale_gbmonth_mean_std_2001_2020.nc')
ds_gpm_gbmonth_mean_std.to_netcdf(scaler_path / 'gpm_imerge_dl_downscale_gbmonth_mean_std_2001_2020.nc')